# Predictive models applied to forecast energy needs in buildings

Author : Charles Gaydon

Started : 29/10/2017

Last Edited : 5/11/2017

### Approach


### Buildings and data

Using Tableau, we saw that we have to predict : 

- Y1, Y2, Y4, Y5 for Bat 1, Bat 2 and Bat 4 ;
- Y1, Y3 and Y5 for Bat 3.

Hence we will use to train :
Bat 1,2,3,4 for Y1, Y5 ;
Bat 1,2,4 for Y2, Y4 ;
Bat 3 for Y3

## Preprocessing

### Import

In [6]:
import numpy as np 
import pandas as pd
import sklearn as sk
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as mpl

X_train = pd.read_csv("./train_set_X.csv",sep=';')
X_test = pd.read_csv("./test_set_X.csv",sep=';')
All_X = pd.concat([X_train,X_test])
All_Y_train = pd.read_csv("./train_set_Y.csv",sep=';')

X_train_size = X_train.shape[0]

### Interpolation
Bat 3 lacks a few data point during a short period of time. 
And when values are equal to an integer 0, it means that they are missing.
We hence choose to interpolate the missing data, which we shall do on X_test as well later. 

In [7]:
from utils import put_nan
All_X.iloc[:,3:] = All_X.iloc[:,3:].applymap(put_nan)
All_Y_train.iloc[:,1:] = All_Y_train.iloc[:,1:].applymap(put_nan)
All_X.interpolate(inplace=True) #TO CHANGE !
#import fancyinput

### Time transformation

Based on this [article](http://www.sciencedirect.com/science/article/pii/S0378778804003032) idea to use hour and date.

In [8]:

#Time transformation
All_X['hour'] = 0
All_X['sinHour'] = 0
All_X['cosHour'] = 0
All_X['sinWeekDay'] = 0
All_X['cosWeekDay'] = 0
All_X["hour"] = All_X['Time'].apply(lambda x : int(x.split()[1][0:2])) #problem here

All_X = All_X.reset_index() #necessary for dayofweek to perform
All_X['Time'] = pd.to_datetime(All_X['Time'])
All_X["weekDay"] = All_X['Time'].dt.dayofweek

All_X['sinHour'] = np.sin(2*np.pi*All_X['hour']/23)
All_X['sinWeekDay'] = np.sin(2*np.pi*All_X['weekDay']/6)
All_X['cosHour'] = np.cos(2*np.pi*All_X['hour']/23)
All_X['cosWeekDay'] = np.cos(2*np.pi*All_X['weekDay']/6)
# we could separate Mon-Fri and Sat-Sun because the dynamics are really different, but we assume
# that adding info about past points is a better alternative.

In [9]:
All_X.head(5)

,index,Id,Id_bat,Time,x1,x2,x3,x4,x5,hour,sinHour,cosHour,sinWeekDay,cosWeekDay,weekDay
0,0,0,1,2016-01-01 00:00:00,1.5,8.1,22.0,380.183,20.0,0,0.000000,1.000000,-0.866025,-0.5,4
1,1,1,1,2016-01-01 01:00:00,1.6,8.2,21.9,378.100,120.0,1,0.269797,0.962917,-0.866025,-0.5,4
2,2,2,1,2016-01-01 02:00:00,1.4,7.9,21.9,374.983,120.0,2,0.519584,0.854419,-0.866025,-0.5,4
3,3,3,1,2016-01-01 03:00:00,1.6,6.9,21.8,376.017,120.0,3,0.730836,0.682553,-0.866025,-0.5,4
4,4,4,1,2016-01-01 04:00:00,1.4,6.3,21.8,376.017,120.0,4,0.887885,0.460065,-0.866025,-0.5,4


### Time dependency

We don't want to use a learning model that is explicitly taking into account past events (e.g. LSTM), instead we add a time dependency by adding sum/peak/value of precedents timestamps

##### Values at t-1 and t-2, max and mean from 12 last observations
We will get rid of the values observations containing values we cannot get.

In [10]:
factors = ['x1','x2','x3','x4','x5']
period = 4
for f in factors :
    for id in range(1,5) : 
        All_X[f+'(t-1)'] = 0
        All_X[f+'(t-2)'] = 0
        All_X[f+'(t-1)'] = All_X[f].shift(1)
        All_X[f+'(t-2)'] = All_X[f].shift(2)
        All_X[f+'max'+str(period)+'H'] =  All_X[f].rolling(period,min_periods=period).max()
        All_X[f+'mean'+str(period)+'H'] =  All_X[f].rolling(period,min_periods=period).mean()
        
    for p in range(1,period+1):
            index = np.where(All_X['Id_bat'] != All_X['Id_bat'].shift(p))[0]
            All_X.loc[index, f]  = np.nan

In [11]:
All_X.dropna(inplace=True)
X_train = All_X.loc[All_X['Id']<list(np.where(All_X['Id_bat'] != All_X['Id_bat'].shift(1)))[0][4]].copy() #to update Y_train
X_test = All_X.loc[All_X['Id']>=list(np.where(All_X['Id_bat'] != All_X['Id_bat'].shift(1)))[0][4]].copy()
Y_train = All_Y_train[(All_Y_train['Id'].isin(X_train['Id'].values))].copy()

We got some missing values in our rolling attribute, an will try to address this issue using the MICE approach, as described [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/)

### Vertical splitting

Missing data in Y : Some data is missing for Bat 3 (i.e. y1) but should be here. We don't want to train on those observations for Y1. So we decide here that we shall separate the prediction of each variable and get rid of only the minimum of observations for each.

In [12]:
#print(All_X_train.describe(), All_Y_train.describe())
print('X :')
print(X_train.shape)
print('Y :')
print(Y_train.shape)

X :
(11676, 35)
Y :
(11676, 6)


In [13]:
Y_div = {}
X_div = {}
X_test_div = {}

### x train
X_div['y1'] = X_train[X_train['Id_bat'].isin([1,2,3,4])].copy()
X_div['y5'] = X_train[X_train['Id_bat'].isin([1,2,3,4])].copy()

X_div['y2']  = X_train[X_train['Id_bat'].isin([1,2,4])].copy()
X_div['y4']  = X_train[X_train['Id_bat'].isin([1,2,4])].copy()

X_div['y3']  = X_train[X_train['Id_bat'].isin([3])].copy()

### x test
X_test_div['y1'] = X_test[X_test['Id_bat'].isin([1,2,3,4])].copy()
X_test_div['y5'] = X_test[X_test['Id_bat'].isin([1,2,3,4])].copy()

X_test_div['y2']  = X_test[X_test['Id_bat'].isin([1,2,4])].copy()
X_test_div['y4']  = X_test[X_test['Id_bat'].isin([1,2,4])].copy()

X_test_div['y3']  = X_test[X_test['Id_bat'].isin([3])].copy()

### y train

Y_div['y1'] = Y_train[X_train['Id_bat'].isin([1,2,3,4])][['Id',"y1"]].copy()
Y_div['y5'] = Y_train[X_train['Id_bat'].isin([1,2,3,4])][['Id',"y5"]].copy()

Y_div['y2'] = Y_train[X_train['Id_bat'].isin([1,2,4])][['Id',"y2"]].copy()
Y_div['y4'] = Y_train[X_train['Id_bat'].isin([1,2,4])][['Id',"y4"]].copy()

Y_div['y3'] = Y_train[X_train['Id_bat'].isin([3])][['Id',"y3"]].copy()

In [14]:
factors = ['y1','y2','y3','y4','y5']

for f in factors:
    print(X_test_div[f].shape)

(2429, 35)
(1784, 35)
(645, 35)
(1784, 35)
(2429, 35)


We remove the last missing value, this time from Y

In [15]:
factors = ['y1','y2','y3','y4','y5']
for f in factors:
    Y_div[f].dropna(inplace=True)
    X_div[f] = X_div[f][X_div[f]['Id'].isin(Y_div[f]['Id'])].copy()
    Y_div[f].drop('Id',axis=1)
    print(X_div[f].shape)
    print(Y_div[f].shape) #OK

(10449, 35)
(10449, 2)
(7917, 35)
(7917, 2)
(2861, 35)
(2861, 2)
(6780, 35)
(6780, 2)
(9352, 35)
(9352, 2)


#### Saving the data to train later

In [17]:
import pickle
output = open('Div_data.pkl','wb')
pickle.dump({'X_div':X_div, 'Y_div':Y_div,'X_test_div':X_test_div},output,-1)
output.close()

### Learning with cross-validation
We want to use five different models for each of our y variable. We generate them using the same template, and will use the pipelines of sklearn to faciliate our work. 
We shall use the RandomForest algo, and use cross-validation (shuffleSplit is enough as the data are approximately well distributed between buildings) to evaluate our model.

In [17]:
from sklearn.ensemble import RandomForestRegressor
n_estimators = 50
def give_me_a_model(style = "RF", params = [n_estimators]):
    if style == "MLP":
        m = MLPRegressor((150,150,150,150),solver ='lbfgs',
            batch_size = 200,max_iter = 100, verbose = True, early_stopping =False)
    elif style == "RF" :
        m = RandomForestRegressor(n_estimators = n_estimators,n_jobs = -1, verbose=0)
    return(m)

In [95]:
#Y_div['y1'].iloc[:,1:].head(2)
#X_div['y1'].head(2)

In [18]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit # we don't really need to stratify by building.
style = "RF"
m = {}
scores = {}
factors = ['y1','y2','y3','y4','y5']
n_splits = 5
for f in factors:  
    m[f] = give_me_a_model(style)
    skf = ShuffleSplit(n_splits=n_splits)
    scores[f] = cross_validate(m[f], X_div[f].iloc[:,4:], np.ravel(Y_div[f].iloc[:,1:]), 
                               scoring = 'neg_mean_squared_error',cv = skf) #, return_train_score = True)
    print('Model for '+f+' has been cross-validated. The '+str(n_splits)+'-MSE is : '+str(-np.mean(scores[f]['test_score'])))
score = 0
for f in factors:
    score+= np.mean(scores[f]['test_score'])
print(-score*n_splits) #19534

Model for y1 has been cross-validated. The 5-MSE is : -366.118555599
Model for y2 has been cross-validated. The 5-MSE is : -1471.00662283
Model for y3 has been cross-validated. The 5-MSE is : -34.491204761
Model for y4 has been cross-validated. The 5-MSE is : -686.204179823
Model for y5 has been cross-validated. The 5-MSE is : -1919.73896433
22387.7976368


### Fitting

In [170]:
for f in factors:  
    m[f] = give_me_a_model(style)
    m[f].fit(X_div[f].iloc[:,4:], np.ravel(Y_div[f].iloc[:,1:]))

### Learning using an off-the-shelve optimizer and RandomForest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

params = {"n_estimators" : list(range(25,150,15))+[175,200]}
par = {}
style = "RF"
m = {}
factors = ['y1','y2','y3','y4','y5']
for f in factors:  
    rfg = RandomForestRegressor(n_jobs=-1)
    m[f] = RandomizedSearchCV(rfg, params, n_jobs=1,refit=True)  
    par[f] = m[f].best_params
    m[f].fit(X_div[f].iloc[:,4:], np.ravel(Y_div[f].iloc[:,1:]))  
    print('Best params found '+f+' with randomized search' + str(par[f]))
    print('Model for '+f+' was fitted and is ready to use. ')

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost.sklearn import XGBClassifier  
from xgboost.sklearn import XGBRegressor

#from sklearn.metrics import mean_squared_error

style = "XGBRegressor"
m = {}
factors = ['y1','y2','y3','y4','y5']
for f in factors:  
    m[f] = XGBRegressor(nthreads=-1)
    
    one_to_left = st.beta(10, 1)  
    from_zero_positive = st.expon(0, 50)
    params = {  
        "n_estimators": st.randint(3, 40),
        "max_depth": st.randint(3, 40),
        "learning_rate": st.uniform(0.05, 0.4),
        "colsample_bytree": one_to_left,
        "subsample": one_to_left,
        "gamma": st.uniform(0, 10),
        'reg_alpha': from_zero_positive,
        "min_child_weight": from_zero_positive,
    }
    
    gs = RandomizedSearchCV(m[f], params, n_jobs=1)  
    gs.fit(X_div[f].iloc[:,4:], np.ravel(Y_div[f].iloc[:,1:]))
    m[f] = gs.best_model_

ModuleNotFoundError: No module named 'xgboost'

### Prediction, shaping and saving

In [ ]:
y_pred = {}
for f in factors:
    y = pd.DataFrame(m[f].predict(X_test_div[f].iloc[:,4:]))
    y['Id'] = X_test_div[f]['Id'].values
    y = y.set_index('Id')
    y_pred[f] = y

In [ ]:
predicted_y = pd.concat(y_pred, axis=1)
predicted_y.columns = factors
predicted_y.insert(0,'Id',predicted_y.index.tolist())
predicted_y.fillna(0,inplace=True)
print(predicted_y.head(5), predicted_y.shape) #OK

predicted_y.to_csv(path_or_buf='RF_with_opti.csv',sep=';',header = True, index=False)

In [ ]:
plot_learning_curve(m24, title, X_train_24, Y_train_24, cv=cv)
title = "Learning Curves)"
plt.show()


In [ ]:
plot_learning_curve(m3, title, X_train_3, Y_train_3, cv=cv)
title = "Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
plt.show()

We test the results.

We generate the prediction for the test set and save the result, before uploading it to the competition's website.